In [1]:
import json
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from tqdm import tqdm
import numpy as np
import scann

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-03-01 13:07:25.857018: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 13:07:25.907289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 13:07:25.908181: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 13:07:26.569761: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

In [3]:
with open('../../examples/relevant_jobs_classification/data/test_converted.json','r') as f:
    data = json.load(f)

In [4]:
# data

In [5]:
df =pd.read_csv('../../examples/relevant_jobs_classification/data/analysis_dataset.csv')

In [6]:
df

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [7]:
df.dtypes

job_id            int64
application_id    int64
user_id           int64
dtype: object

In [8]:
jid_list= [1375026,1374956,1373879,1374981,1374473,1376033,1375642,1373929,1373732,1373755]

In [9]:
len(jid_list)

10

In [10]:
df = df.loc[df['job_id'].isin(jid_list)]

In [11]:
df.job_id.nunique()

10

In [12]:
df

,job_id,application_id,user_id
2785,1373732,144320806,209995
2786,1373732,144315225,882096
2787,1373732,144309063,761462
2788,1373732,144305516,2749339
2789,1373732,144293774,1227
...,...,...,...
132314,1376033,144435189,1968534
132758,1373755,144435975,2662357
133436,1376033,144437077,1032543
133588,1376033,144437341,788410


In [13]:

df.user_id.nunique()

476

In [14]:
df['user_id']=df['user_id'].astype(int)

/tmp/ipykernel_7270/2460164112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['user_id']=df['user_id'].astype(int)


In [15]:
df.dtypes

job_id            int64
application_id    int64
user_id           int64
dtype: object

In [16]:
# uid_list = list(data.keys())[:5]

In [17]:
# data

In [18]:
df_list = []

for id, desc in tqdm(data.items()):
    try:
        descriptions = "; ".join([exp['description'] for exp in desc])
        df_list.append({'id': id, 'descriptions': descriptions})
    except Exception as e:
        # print(e)
        continue

udf = pd.DataFrame(df_list)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 389/389 [00:00<00:00, 467368.74it/s]


In [19]:
udf

,id,descriptions
0,209995,Initiating Coverage and tracking companies wit...
1,882096,Tracking Banking sector with 25 stocks uneer a...
2,761462,Part of 4-member investment team of a sector a...
3,2749339,Independently raised INR 10 Cr for early-stage...
4,1227,Setting up the Family office / Treasury Head /...
...,...,...
302,2700303,Preparing Job descriptions in consultation wit...
303,1968051,Continuous monitoring and preparation of COP w...
304,2754531,Zero Based Budgeting and Overhead control- Act...
305,2753323,Oversaw sales and marketing in entire North In...


In [20]:
udf['id']=udf['id'].astype(int)

In [21]:
# df[1:2].descriptions.values

In [22]:
# data['2673538']

In [23]:
udf.dtypes

id               int64
descriptions    object
dtype: object

In [24]:
udlist=list(udf.id)

In [25]:
len(udlist)

307

In [26]:
df.loc[df['user_id'].isin(udlist)].user_id.nunique()

307

In [27]:
df = df.loc[df['user_id'].isin(udlist)]

In [28]:
df.job_id.nunique()

10

In [29]:
df.user_id.nunique()

307

In [80]:
import requests

def fetch_jobs(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    

    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    # print(response.text)
    

    # url = url.replace("<DAYS>",str(last_k)).replace("<SIZE>",str(size))

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()
    # print(job_data)
    job_df = pd.DataFrame(job_data)

    return job_df[['id','introText']]

In [31]:
from tqdm import tqdm
df_list = []
for i in tqdm(df['job_id'].unique()):
    try:
        j = fetch_jobs(str(i))
    
        df_list.append(j)
    except:
        continue
jd = pd.concat(df_list) 

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 28.37it/s]


In [32]:
jid_list = list(jd.id)

In [33]:
len(jid_list)

9

In [34]:
df1 = df.loc[df['job_id'].isin(jid_list)]

In [35]:
df1.shape

(299, 3)

In [36]:
jd.columns = ['job_id','introText']

In [37]:
merged_df = pd.merge(jd, df1, on='job_id', how='left')


In [38]:
merged_df.shape

(299, 4)

In [42]:
uid_list = list(merged_df.user_id.unique())

In [43]:
merged_df.user_id.nunique()

265

In [44]:
len(uid_list)

265

In [45]:
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string
     

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)

    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()
    
            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'resume' : []
    }

    for s in tqdm(user_json):

        try:
            source = s[0]

            user_dict['id'].append(source['id'])
            user_dict['resume'].append(clean_space(source['resumeText']))

        except Exception as e:
            continue

    return user_dict

In [46]:

def get_user_data_in_batches(uid_list, batch_size=1000):
    user_data = pd.DataFrame()

    for i in range(0, len(uid_list), batch_size):
        batch_uids = uid_list[i:i + batch_size]

        batch_user_data = get_user_data_search_embed(batch_uids)

        user_data = pd.concat([user_data, batch_user_data], ignore_index=True)

    return user_data

In [47]:
batched_user_data = get_user_data_in_batches(uid_list, batch_size=1000)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 265/265 [00:00<00:00, 4678.62it/s]


In [48]:
batched_user_data.columns = ['user_id','resume']

In [49]:
batched_user_data['user_id']=batched_user_data['user_id'].astype(int)

In [50]:
main_df=pd.merge(merged_df,batched_user_data, on='user_id', how='left')

In [52]:
main_df.job_id.nunique()

9

In [53]:
main_df.user_id.nunique()

265

In [54]:
def clean_text(input_text):
    
    '''Make text lowercase, remove text in square brackets,remove links,
    remove punctuation and remove words containing numbers.'''

    # This code block was added to remove the company names from Job embeddings
    
    # t = input_text.split('-')
    # if len(t)>3:
    #     t1 = t[1:]
    #     text = ' '.join(t1)
    # else:
    #     text = input_text
    #     pass

    text = input_text
        
    text = str(text).lower()
    text = re.sub('\[.*?\]',' ', text)
    text = re.sub('https?://\S+|www\.\S+',' ', text)
    text = re.sub('<.*?>+',' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation),' ', text)
    text = re.sub('\n',' ', text)
    text = re.sub('\w*\d\w*',' ', text)
    text = re.sub('yrs',' ',text)
    text = re.sub('\r',' ',text)
    text = ' '.join([word for word in text.split() if word not in (stop)])
    text = re.sub(' +', ' ', text)
    
    return text

In [55]:
li = []
for d in tqdm(main_df['introText']):
    des = clean_text(d)
    li.append(des)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 299/299 [00:00<00:00, 1754.63it/s]


In [56]:
main_df['description']=li

In [57]:
li = []
for r in tqdm(main_df['resume']):
    res = clean_text(r)
    li.append(res)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 299/299 [00:00<00:00, 574.44it/s]


In [58]:
main_df['resume_text']=li

In [59]:
mdf = main_df[['job_id','description','user_id','resume_text']]

In [61]:
mdf.job_id.nunique()

9

In [62]:
mdf.user_id.nunique()

265

In [63]:
mdf.job_id.value_counts()

job_id
1373879    43
1375642    39
1373732    37
1376033    37
1373929    33
1374981    31
1375026    29
1374956    27
1373755    23
Name: count, dtype: int64

In [64]:
import numpy as np
from tqdm import tqdm


batch_size = 15  # Adjust the batch size according to your system's capacity

df_list = []

unique_job_ids = mdf.job_id.unique()

num_batches = len(unique_job_ids) // batch_size + 1

for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_job_ids = unique_job_ids[start_idx:end_idx]

    batch_df_list = []

    try:

        for jid in batch_job_ids:
            jobdf = mdf.loc[mdf['job_id'] == jid]
    
            description = jobdf.description.sample().values[0]
            job_embeddings = model.encode([description])[0]
    
            udf = jobdf[['user_id', 'resume_text']]
            udf_list = udf.values
            r_list = udf_list[:, 1].tolist()
            uid_list = udf_list[:, 0].tolist()
            
            try:
                embedding_sublist = [e for e in model.encode(r_list)]
            except Exception as e:
                # print(uid_list)
                print('problem')
                continue
    
            sub_dict = {uid: np.array([embedding]) for uid, embedding in zip(uid_list, embedding_sublist)}
    
            uv = embedding_sublist
    
            try:
                nu = uv / np.linalg.norm(uv, axis=1)[:, np.newaxis]
            except Exception as e:
                print(f'Error is {e}')
    
            searcher = scann.scann_ops_pybind.builder(nu, 43, "dot_product").score_brute_force(2).build()

        #     searcher = scann.scann_ops_pybind.builder(nu, 900, "dot_product").tree(
        # num_leaves=10, num_leaves_to_search=10, training_sample_size=1000).score_ah(
        # 2, anisotropic_quantization_threshold=0.2).reorder(1000).build()

    
            neighbours, distances = searcher.search_batched([job_embeddings])
    
            udf = udf.iloc[neighbours[0]]
            udf['scores'] = distances[0]
            udf.drop(['resume_text'], axis=1, inplace=True)
    
            final_df = pd.merge(jobdf, udf, on='user_id', how='left')
            batch_df_list.append(final_df)            
            
            del jobdf, udf, embedding_sublist, nu, searcher

    
        batch_result_df = pd.concat(batch_df_list)
        df_list.append(batch_result_df)
    except Exception as e:
        print(e)
        continue
result_df = pd.concat(df_list)


  0%|                                                                                                                                                  | 0/1 [00:00<?, ?it/s][libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannC

In [65]:
result_df.job_id.nunique()

9

In [66]:
result_df.user_id.nunique()

265

In [67]:
result_df.job_id.sample(5)

15    1373732
3     1373879
22    1373879
0     1376033
9     1373732
Name: job_id, dtype: int64

In [74]:
result_df

,job_id,description,user_id,resume_text,scores
0,1373732,mumbai based family office looking hire equity...,209995,akshay pradhan address neelkanth sagar mahatma...,0.645710
1,1373732,mumbai based family office looking hire equity...,882096,name mayank kumar gulgulia tel summary cfa usa...,0.690789
2,1373732,mumbai based family office looking hire equity...,761462,sahil nandkumar cfa date birth address flat se...,0.673624
3,1373732,mumbai based family office looking hire equity...,2749339,kushboo jaichand luniya luniya kushboo gmail c...,0.618894
4,1373732,mumbai based family office looking hire equity...,1227,sachin jain mba finance city mumbai contact gm...,0.688748
...,...,...,...,...,...
32,1376033,agm sales need able create vision area execute...,2492721,nikhil kumar citizenship indian date birth may...,0.497188
33,1376033,agm sales need able create vision area execute...,1205250,krishna kant tiwari male years gmail com summa...,0.662455
34,1376033,agm sales need able create vision area execute...,1895984,rajesh yadav rajesh fms edu p account manager ...,0.577546
35,1376033,agm sales need able create vision area execute...,2753323,kapil ahuja zonal sales head pharma diagnostic...,0.651153


In [68]:
result_df.loc[result_df['job_id']==1376033].sort_values(['scores'],ascending=False)

,job_id,description,user_id,resume_text,scores
6,1376033,agm sales need able create vision area execute...,524961,bhuwan gaur e mail bhuwan gaur gmail com phone...,0.705685
36,1376033,agm sales need able create vision area execute...,2749129,adarsh singh adarsh yahoo com kolkata india ww...,0.668222
2,1376033,agm sales need able create vision area execute...,1780851,ashish sharma people nd upbeat team player rob...,0.664795
21,1376033,agm sales need able create vision area execute...,907959,sumit arora regency park dlf phase gurugram ph...,0.663189
33,1376033,agm sales need able create vision area execute...,1205250,krishna kant tiwari male years gmail com summa...,0.662455
22,1376033,agm sales need able create vision area execute...,1186436,profile summary orchestrated transformative st...,0.652720
35,1376033,agm sales need able create vision area execute...,2753323,kapil ahuja zonal sales head pharma diagnostic...,0.651153
10,1376033,agm sales need able create vision area execute...,218753,amar nath lalka male dob aug yahoo co executiv...,0.632726
17,1376033,agm sales need able create vision area execute...,724066,sachin chauhan performance driven professional...,0.630560
26,1376033,agm sales need able create vision area execute...,1409593,page operations attrition recruitment training...,0.620907


In [70]:
# result_df.to_csv('../../examples/relevant_jobs_classification/data/score_result_v1.csv',index=False)

[1373732,
 1373755,
 1373879,
 1373929,
 1374956,
 1374981,
 1375026,
 1375642,
 1376033]

In [72]:

result_df.job_id.unique()

array([1373732, 1373755, 1373879, 1373929, 1374956, 1374981, 1375026,
       1375642, 1376033])

In [73]:
# jid_list= [1375026,1374956,1373879,1374981,1374473,1376033,1375642,1373929,1373732,1373755]

In [79]:
result_df.job_id.sample().unique()[0]

1376033

In [84]:
def do_something():
    jid = result_df.job_id.sample().unique()[0]
    job_df = fetch_jobs(str(jid))
    uid_list = result_df.loc[result_df['job_id']==jid].user_id
    return job_df

In [87]:
d = do_something()

In [88]:
d

,id,title,tags
0,1374956,Director - Education & Government Consulting (...,"[{'id': 625, 'name': 'Consulting - eGov/PSU', ..."


In [90]:
list(result_df.loc[result_df['job_id']==jid].user_id)

[238941,
 1723856,
 1780851,
 1533484,
 1286631,
 89101,
 524961,
 285310,
 545221,
 972268,
 218753,
 157639,
 2462583,
 1480843,
 1181133,
 289276,
 161832,
 724066,
 7518,
 1648670,
 1660316,
 907959,
 1186436,
 107655,
 567843,
 1918487,
 1409593,
 44472,
 2265321,
 598438,
 68358,
 2789031,
 2492721,
 1205250,
 1895984,
 2753323,
 2749129]